In [1]:
# !pip install --upgrade langchain accelerate transformers sentencepiece
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [7]:
from langchain.chains import (
    LLMChain,
    SequentialChain
)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

In [21]:
# Globals
repo_id = 'google/flan-t5-xxl'

tokenizer = T5Tokenizer.from_pretrained(repo_id)
model = T5ForConditionalGeneration.from_pretrained(repo_id, device_map="auto", offload_folder="/tmp/offload")

generate_text = pipeline(
    "text2text-generation",
    max_length=1000,
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
)

transformer_pipeline = HuggingFacePipeline(pipeline=generate_text)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
prompt = PromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain = LLMChain(
    llm=transformer_pipeline,
    prompt=prompt
)

In [24]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)

a designer


---

## SequentialChain

In [26]:
# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm=transformer_pipeline, prompt=first_prompt,
    output_key="english_review"
)

second_prompt = PromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence using your own words:"
    "\n\n{english_review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm=transformer_pipeline, prompt=second_prompt,
    output_key="summary"
)

# prompt template 3: translate to english
third_prompt = PromptTemplate.from_template(
    "What language is the following review:\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=transformer_pipeline, prompt=third_prompt,
    output_key="language"
)

# prompt template 4: follow up message
fourth_prompt = PromptTemplate.from_template(
    "Is the sentiment in the summary Positive or Negative?"
    "\n\nFeedback: {summary}"
)
# chain 4: input= summary, language and output= sentiment
chain_four = LLMChain(
    llm=transformer_pipeline, prompt=fourth_prompt,
    output_key="sentiment"
)

In [27]:
# overall_chain: input= review 
# and output= language, english_review,summary, sentiment
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["language", "english_review", "summary", "sentiment"],
    verbose=False
)

spanish_review = "Encuentro el sabor mediocre. La espuma no aguanta, es raro. Compro lo mismo en el mercado y el sabor es mucho mejor...\n¿Lote antiguo o falsificado?"

result = overall_chain(spanish_review)
print(result)

{'review': 'Encuentro el sabor mediocre. La espuma no aguanta, es raro. Compro lo mismo en el mercado y el sabor es mucho mejor...\n¿Lote antiguo o falsificado?', 'language': 'Spanish', 'english_review': 'I find the taste mediocre. The foam does not last, it is rare. Buy the same in the market and the taste is much better... Old lot or fake?', 'summary': 'I find the taste mediocre. The foam does not last, it is rare. Buy the same in the market and the taste is much better... Old lot or fake?', 'sentiment': 'Negative'}
